In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
questions = pd.read_csv('../input/riiid-test-answer-prediction/questions.csv',                         
                        usecols=[0, 1, 2, 3, 4],
                           dtype={'question_id': 'int16',
                                  'part': 'int8',
                                  'bundle_id': 'int8',
                                  'correct_answer': 'int8',
                                  'tags': 'object'}
                          )


df_train = pd.read_csv('../input/riiid-test-answer-prediction/train.csv',
                   usecols=[0, 1, 2, 3, 4, 5, 7, 8, 9],
                   dtype={
                          'row_id': 'int32',
                       'timestamp': 'int64',
                          'user_id': 'int32',
                          'content_id': 'int16',
                          'content_type_id': 'int8',
                          'task_container_id': 'int16',
                          'user_answer': 'int8',
                          'answered_correctly':'int8',
                          'prior_question_elapsed_time': 'float32',
                          'prior_question_had_explanation': 'boolean'
                          },
                   )

print(df_train.shape)

df_train = df_train[df_train.answered_correctly!= -1 ]
df_train = df_train[df_train['content_type_id'] == 0]

df_train.drop(['content_type_id'], axis=1, inplace=True)

print(df_train.shape)

df_train.dropna(inplace=True)

df_train = df_train.drop_duplicates(subset=['user_id', 'content_id'], keep='first')

print(df_train.shape)

In [ ]:
# sort rows by timestamp in each item
df_train['prior_question_elapsed_time_shift'] = df_train.sort_values(by = 'timestamp').groupby('user_id')['prior_question_elapsed_time'].shift(-1)
df_train.dropna(inplace=True)

In [ ]:
#remove outliers for each question
import tqdm

time_upper_bound = df_train.groupby(['content_id'])['prior_question_elapsed_time_shift'].quantile(0.99)
time_lower_bound = df_train.groupby(['content_id'])['prior_question_elapsed_time_shift'].quantile(0.01)

include = []
for row in tqdm.tqdm(df_train.itertuples(),total=df_train.shape[0]):

    if row.timestamp <= time_upper_bound[row.content_id] and row.timestamp >= time_lower_bound[row.content_id]:
        include.append(True)
    else:
        include.append(False)

df_train['include'] = include
df_train = df_train[df_train['include']==True]

df_train.shape

In [ ]:
## keep sample size >20 
tmp = df_train['content_id'].value_counts().reset_index()
question_id_gt_20 = tmp[tmp['count'] > 20]['content_id'].tolist()
df_train = df_train[df_train['content_id'].isin(question_id_gt_20)]
df_train.shape

In [ ]:
print(df_train['content_id'].value_counts(), df_train['user_id'].value_counts())

In [ ]:
# prepare data for LMM model
Popn1 = df_train.groupby('content_id', sort=False).size().reset_index()
Popn1.to_csv('/kaggle/working/Popn1.csv')

item_diff = df_train.groupby('content_id', sort=False, as_index=False)['answered_correctly'].mean().reset_index()
df_train = df_train.merge(item_diff, how='inner', left_on='content_id', right_on='content_id')
data_pop = df_train.rename(columns = {'answered_correctly_y': 'item_diff'})

Xmean_df = data_pop.groupby('content_id', sort=False)[['item_diff']].mean().reset_index()
Xmean_df.to_csv('/kaggle/working/Xmean1.csv')

In [ ]:
# split train, val and test data

fraction = 'first_half'
train_ = df_train.groupby('index').apply(lambda x: x.iloc[:x.content_id.size//2]) #the first half of rows within each item
val_ = df_train.groupby('index').apply(lambda x: x.iloc[x.content_id.size//2:(x.content_id.size//2 + x.content_id.size//4)]) #the first half of rows within each item
test_ = df_train.groupby('index').apply(lambda x: x.iloc[(x.content_id.size//2 + x.content_id.size//4):]) #the first half of rows within each item


# train_.to_csv('/kaggle/working/train_{}.csv'.format(fraction))
# val_.to_csv('/kaggle/working/val_{}.csv'.format(fraction))
# test_.to_csv('/kaggle/working/test_{}.csv'.format(fraction))
# train_.groupby('content_id')['prior_question_elapsed_time_shift'].quantile(q=0.95).reset_index().to_csv('/kaggle/working/time_95_{}.csv'.format(fraction))

In [ ]:
print(train_['content_id'].value_counts(), train_['user_id'].value_counts().size)
print(val_['content_id'].value_counts(), val_['user_id'].value_counts().size)
print(test_['content_id'].value_counts(), test_['user_id'].value_counts().size)

### regression

In [ ]:
train_full = train_.merge(questions, how='inner', left_on='content_id', right_on='question_id')
val_full = val_.merge(questions, how='inner', left_on='content_id', right_on='question_id')
test_full = test_.merge(questions, how='inner', left_on='content_id', right_on='question_id')

In [ ]:
## find the 95th RT in each item and use it as dependent variable in regression, merge with question feature

train_reg = train_.groupby('content_id')['prior_question_elapsed_time_shift'].quantile(q=0.95).reset_index()
test_reg_n = train_.groupby('content_id').size().reset_index() #add n
train_reg = train_reg.merge(test_reg_n, how='inner', on='content_id')

train_reg = train_reg.merge(questions, how='inner', left_on='content_id', right_on='question_id')
train_reg.rename(columns = {'prior_question_elapsed_time_shift': 'time_q95', 0: 'n'}, inplace = True)

val_reg = val_.groupby('content_id')['prior_question_elapsed_time_shift'].quantile(q=0.95).reset_index()
val_reg = val_reg.merge(train_reg[['content_id', 'n']], how='inner', on='content_id' )
val_reg = val_reg.merge(questions, how='inner', left_on='content_id', right_on='question_id')
val_reg.rename(columns = {'prior_question_elapsed_time_shift': 'time_q95'}, inplace = True)

test_reg = test_.groupby('content_id')['prior_question_elapsed_time_shift'].quantile(q=0.95).reset_index()
test_reg = test_reg.merge(train_reg[['content_id', 'n']], how='inner', on='content_id' )
test_reg = test_reg.merge(questions, how='inner', left_on='content_id', right_on='question_id')
test_reg.rename(columns = {'prior_question_elapsed_time_shift': 'time_q95'}, inplace = True)

In [ ]:
train_reg.shape, test_reg.shape, val_reg.shape

In [ ]:
train_reg.to_csv('/kaggle/working/train_reg_{}.csv'.format(fraction))
test_reg.to_csv('/kaggle/working/test_reg_{}.csv'.format(fraction))
val_reg.to_csv('/kaggle/working/val_reg_{}.csv'.format(fraction))

In [ ]:
# test_reg = pd.read_csv('/kaggle/input/data-regg/test_reg_{}.csv'.format(fraction))

# #test_reg = test_reg.iloc[int(test_reg.shape[0]//2):,]
# val_reg = pd.read_csv('/kaggle/input/data-regg/val_reg_{}.csv'.format(fraction))
# #val_reg = test_reg.iloc[:int(test_reg.shape[0]//2),]

# train_reg = pd.read_csv('/kaggle/input/data-regg/train_reg_{}.csv'.format(fraction))
# Xmean1 = pd.read_csv('/kaggle/input/data-regg/Xmean1.csv')


In [ ]:
# include item difficulty variable
Xmean1 = pd.read_csv('/kaggle/working/Xmean1.csv') #Xmean_df above

train_reg = train_reg.merge(Xmean1, how='inner', left_on='content_id', right_on='content_id')
val_reg = val_reg.merge(Xmean1, how='inner', left_on='content_id', right_on='content_id')
test_reg = test_reg.merge(Xmean1, how='inner', left_on='content_id', right_on='content_id')

train_reg.shape, test_reg.shape, val_reg.shape

In [ ]:
## creating tag feature for regression
all_tags = []
for t in train_reg['tags'].tolist():
    all_tags.extend(t.split())
    
all_tags = np.unique(all_tags)
def create_tag_df(df):
    
    tags_array = np.array([[0] * len(all_tags)] * df.shape[0])
    for r, tags in enumerate(df['tags'].tolist()):
        tags_array[r, [i for i, e in enumerate(all_tags) if e in tags.split()]] = 1

    df_tags = pd.DataFrame(tags_array)

    D = {}
    for e, i in enumerate(df_tags.columns):
        D[i] = 'tag_' + all_tags[e]

    df_tags_ = df_tags.rename(columns= D)
    df_tags_['content_id'] = df['content_id']
    df = df.merge(df_tags_, on='content_id')
    
    return df

train_reg = create_tag_df(train_reg)
val_reg = create_tag_df(val_reg)
test_reg = create_tag_df(test_reg)

train_reg.shape, test_reg.shape, val_reg.shape

In [ ]:
train_reg.head()

In [ ]:
# train_reg = train_reg[train_reg.bundle_id!= -10 ]
# val_reg = val_reg[val_reg.bundle_id!= -10 ]
# test_reg = test_reg[test_reg.bundle_id!= -10 ]

train_reg['part'] = train_reg['part'].astype('category')
val_reg['part'] = val_reg['part'].astype('category')
test_reg['part'] = test_reg['part'].astype('category')

train_reg['bundle_id'] = train_reg['bundle_id'].astype('category')
val_reg['bundle_id'] = val_reg['bundle_id'].astype('category')
test_reg['bundle_id'] = test_reg['bundle_id'].astype('category')

train_reg.shape, test_reg.shape, val_reg.shape

In [ ]:
# small sample data with train record <50 rows each item
train_reg_small = train_reg[train_reg['n']<50]
train_reg['n'].max()

In [ ]:
### weighted least square

from sklearn import linear_model, svm
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import sklearn.metrics as metrics

tags_col_names = train_reg.columns.tolist()[10:]
### weighted ls regression
train_reg_small = train_reg[train_reg['n']<50] #or <50, 70305 is the max n for full data
val_reg_small = val_reg[val_reg['content_id'].isin(train_reg_small['content_id'].tolist())]
test_reg_small = test_reg[test_reg['content_id'].isin(train_reg_small['content_id'].tolist())]

X_train = train_reg_small[['bundle_id', 'part', 'item_diff']]#+ tags_col_names]
y_train =train_reg_small[['time_q95']]
X_val = val_reg_small[['bundle_id', 'part', 'item_diff']]#+ tags_col_names]
y_val =val_reg_small[['time_q95']]
X_test = test_reg_small[['bundle_id', 'part', 'item_diff']]#+ tags_col_names]
y_test = test_reg_small[['time_q95']]

regr = LinearRegression()
regr.fit(X_train, np.log(y_train))

y_pred = regr.predict(X_train)
residual = (y_train - np.exp(y_pred))
residual = np.array(residual['time_q95'].tolist())

regr = LinearRegression()
regr.fit(X_train, np.log(y_train), 1/np.sqrt(residual**2))
y_pred = regr.predict(X_test)

explained_variance=metrics.explained_variance_score(y_test, np.exp(y_pred))
mean_absolute_error=metrics.mean_absolute_error(y_test, np.exp(y_pred)) 
mse=metrics.mean_squared_error(y_test, np.exp(y_pred)) 
mean_squared_log_error=metrics.mean_squared_log_error(y_test, np.exp(y_pred))
median_absolute_error=metrics.median_absolute_error(y_test, np.exp(y_pred))
r2=metrics.r2_score(y_test, y_pred)

print('explained_variance: ', round(explained_variance,4))    
print('mean_squared_log_error: ', round(mean_squared_log_error,4))
print('r2: ', round(r2,4))
print('MAE: ', round(mean_absolute_error,4))
print('MSE: ', round(mse,4))
print('RMSE: ', round(np.sqrt(mse),4))

In [ ]:
test_reg_small['time_pred'] = np.exp(y_pred)

ratio = test_reg_small['time_pred'] / test_reg_small['time_q95'] #final_time_est

ra = []
for i in ratio.tolist():
    if i > 2:
        ra.append('>2')
    elif i <= 2 and i > 1.5:
        ra.append('1.5-2')
    elif i <= 1.5 and i > 1.2:
        ra.append('1.2-1.5')
    elif i <= 1.2 and i > 1:
        ra.append('1-1.2')
    elif i <= 1 and i > 0.8:
        ra.append('0.8-1')
    elif i <= 0.8 and i > 0.5:
        ra.append('0.5-0.8')
    else:
        ra.append('0-0.5')

test_reg_small['ra'] = ra
print(test_reg_small['ra'].value_counts())

''' regression full 
ra
0.8-1      1495
1-1.2      1357
1.2-1.5     944
0.5-0.8     657
1.5-2       205
>2           43
0-0.5        11
Name: count, dtype: int64
'''

#regression small 
# ra
# 0.8-1      1149
# 1-1.2      1147
# 1.2-1.5     840
# 0.5-0.8     544
# 1.5-2       203
# >2           22
# 0-0.5         9

In [ ]:
np.std(np.abs(test_reg_small['time_pred'] - test_reg_small['time_q95']))

In [ ]:
np.std(np.abs(test_reg_small['time_pred'] - test_reg_small['time_q95']))

In [ ]:
X_train.shape

In [ ]:
## xgboost

from sklearn.model_selection import cross_val_score, PredefinedSplit
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

# define model
model = GradientBoostingRegressor()

parameters = {
    'max_depth': [2,3,4,5,6,7,8,9,10],
    'n_estimators': [30, 60, 100, 120],
    'learning_rate': [0.01, 0.03, 0.05,0.07, 0.09]
}


#cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
ps = PredefinedSplit(np.concatenate([np.zeros(X_train.shape[0])-1, np.zeros(X_val.shape[0])]))
grid_search = GridSearchCV(
    estimator=model,
    param_grid=parameters,
    scoring = 'neg_mean_absolute_error',
    n_jobs = 10,
    cv = ps,
    verbose=1
)

grid_search.fit(pd.concat([X_train, X_val]), np.log(np.array(pd.concat([y_train['time_q95'], y_val['time_q95']]))))

grid_search.best_estimator_


#n_scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
#print('MAE: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

In [ ]:
model = GradientBoostingRegressor(max_depth=7, n_estimators=120, learning_rate=0.07)

model.fit(X_train,np.array(y_train['time_q95']))
y_pred_gbr = model.predict(X_test)

In [ ]:
test_reg_small['time_pred'] = y_pred_gbr

ratio = test_reg_small['time_pred'] / test_reg_small['time_q95'] #final_time_est

ra = []
for i in ratio.tolist():
    if i > 2:
        ra.append('>2')
    elif i <= 2 and i > 1.5:
        ra.append('1.5-2')
    elif i <= 1.5 and i > 1.2:
        ra.append('1.2-1.5')
    elif i <= 1.2 and i > 1:
        ra.append('1-1.2')
    elif i <= 1 and i > 0.8:
        ra.append('0.8-1')
    elif i <= 0.8 and i > 0.5:
        ra.append('0.5-0.8')
    else:
        ra.append('0-0.5')

test_reg_small['ra'] = ra
print(test_reg_small['ra'].value_counts())

''' full
ra
1-1.2      1725
0.8-1      1469
1.2-1.5     893
0.5-0.8     401
1.5-2       203
>2           13
0-0.5         8
Name: count, dtype: int64
''' 
#small
# ra
# 1-1.2      1322
# 0.8-1      1156
# 1.2-1.5     799
# 0.5-0.8     368
# 1.5-2       241
# >2           22
# 0-0.5         6

In [ ]:
pd.DataFrame({'y_red_reg': np.exp(y_pred).reshape(-1), 'y_red_gbr': y_pred_gbr}).to_csv('gbr_reg_small_pred_notag2.csv')

In [ ]:
np.std(np.abs(test_reg_small['time_pred'] - test_reg_small['time_q95']))

in Rstudio, run the code of EBLUP
```
library('lme4')
library('sae')
library('dplyr')

df_test_ = read.csv('test_first_half.csv')
df_val_ = read.csv('val_first_half.csv')
df_train_ = read.csv('train_first_half.csv')
train_reg = read.csv('train_reg_first_half.csv')
test_reg = read.csv('test_reg_first_half.csv')
val_reg = read.csv('val_reg_first_half.csv')

Popn1 = read.csv('Popn1.csv')
time_95 = read.csv('time_95_first_half.csv')
Xmean1 = read.csv('Xmean1_.csv')

df_val_ = val_reg
df_test_ = test_reg

###### EBLUP
df_train_$prior_question_elapsed_time_shift_log = log(df_train_$prior_question_elapsed_time_shift+0.01)
elbupResults = eblupBHF(prior_question_elapsed_time_shift_log~ answered_correctly_x, 
                        dom=content_id, meanxpop=data.frame(Xmean1$content_id, Xmean1$item_diff), popnsize=Popn1, data=df_train_)

results_df = df_train_ %>% group_by(content_id) %>% summarize(Mean = mean(prior_question_elapsed_time_shift, na.rm=TRUE),
                                                       Quant95 = quantile(prior_question_elapsed_time_shift, c(0.95)),
                                                       sd = sd(prior_question_elapsed_time_shift, na.rm=TRUE), 
                                                       n = n(),
                                                       mse = sd(prior_question_elapsed_time_shift, na.rm=TRUE) / sqrt(n()))

results_eblup  = data.frame (content_id  = unique(df_train_$content_id),
                             eblup = exp(elbupResults$eblup$eblup)
)

########## merge and validate
jointdataset <- merge(results_df, results_eblup, by = 'content_id')
#val_time_95 = df_val_ %>% group_by(content_id) %>% summarize(Quant95 = quantile(prior_question_elapsed_time_shift, c(0.95)))

freqs = NULL
for(z in  seq(1.5, 2, by=0.01)){
  final_time_est = jointdataset$eblup + z*jointdataset$sd #j #1.62*jointdataset$mse*sqrt(jointdataset$n) #
  
  jointdataset$final_time_est = final_time_est
  results_full = merge(jointdataset, df_val_, by = 'content_id')
  
  results_full <- na.omit(results_full)
  #results_full = results_full[results_full$n>50,]
  
  ratio_95 = results_full['Quant95'] / results_full['time_q95'] #final_time_est
  ratio_re = results_full['final_time_est'] / results_full['time_q95'] #final_time_est
  
  ra_95 = NULL
  for(i in ratio_95$Quant95){
    if(i>2){
      ra_95 = c(ra_95, '>2')
    }else if(i <= 2 && i > 1.5){
      ra_95 = c(ra_95, '1.5-2')
    }else if(i <= 1.5 && i > 1.2){
      ra_95 = c(ra_95, '1.2-1.5')
    }else if(i <= 1.2 && i > 1){
      ra_95 = c(ra_95, '1-1.2')
    }else if(i <= 1 && i > 0.8){
      ra_95 = c(ra_95, '0.8-1')
    }else if(i <= 0.8 && i > 0.5){
      ra_95 = c(ra_95, '0.5-0.8')
    }else{
      ra_95 = c(ra_95, '0-0.5')
    }
  }
  
  ra_re = NULL
  for(i in ratio_re$final_time_est){
    
    if(i>2){
      ra_re = c(ra_re, '>2')
    }else if(i <= 2 && i > 1.5){
      ra_re = c(ra_re, '1.5-2')
    }else if(i <= 1.5 && i > 1.2){
      ra_re = c(ra_re, '1.2-1.5')
    }else if(i <= 1.2 && i > 1){
      ra_re = c(ra_re, '1-1.2')
    }else if(i <= 1 && i > 0.8){
      ra_re = c(ra_re, '0.8-1')
    }else if(i <= 0.8 && i > 0.5){
      ra_re = c(ra_re, '0.5-0.8')
    }else{
      ra_re = c(ra_re, '0-0.5')
    }
  }
  
  freq = as.data.frame(table(ra_re))$Freq 
  
  freqs = cbind(freqs,  freq)
  
}

results = cbind(as.data.frame(table(ra_95)), freqs)

re_freq_names = NULL
for(n in seq(1.5, 2, by=0.01)){
  re_freq_names = c(re_freq_names, paste0('freq', n))
}

colnames(results) = c('ra_95', 'Freq', re_freq_names)

############## test
#test_time_95 = df_test_ %>% group_by(content_id) %>% summarize(Quant95 = quantile(prior_question_elapsed_time_shift, c(0.95), na.rm=TRUE))
##select(df_test_, content_id, time_q95) #
test_time_95 = df_test_[,c('content_id', 'time_q95')]
final_time_est = jointdataset$eblup + 1.78*jointdataset$sd #j #1.62*jointdataset$mse*sqrt(jointdataset$n) #

jointdataset$final_time_est = final_time_est
results_full = merge(jointdataset, test_time_95, by = 'content_id')

#results_full <- na.omit(results_full)
results_full = results_full[results_full$n<50,]

ratio_95 = results_full['Quant95'] / results_full['time_q95'] #final_time_est
ratio_re = results_full['final_time_est'] / results_full['time_q95'] #final_time_est

ra_95 = NULL
for(i in ratio_95$Quant95){
  if(i>2){
    ra_95 = c(ra_95, '>2')
  }else if(i <= 2 && i > 1.5){
    ra_95 = c(ra_95, '1.5-2')
  }else if(i <= 1.5 && i > 1.2){
    ra_95 = c(ra_95, '1.2-1.5')
  }else if(i <= 1.2 && i > 1){
    ra_95 = c(ra_95, '1-1.2')
  }else if(i <= 1 && i > 0.8){
    ra_95 = c(ra_95, '0.8-1')
  }else if(i <= 0.8 && i > 0.5){
    ra_95 = c(ra_95, '0.5-0.8')
  }else{
    ra_95 = c(ra_95, '0-0.5')
  }
}

ra_re = NULL
for(i in abs(ratio_re$final_time_est)){
  
  if(i>2){
    ra_re = c(ra_re, '>2')
  }else if(i <= 2 && i > 1.5){
    ra_re = c(ra_re, '1.5-2')
  }else if(i <= 1.5 && i > 1.2){
    ra_re = c(ra_re, '1.2-1.5')
  }else if(i <= 1.2 && i > 1){
    ra_re = c(ra_re, '1-1.2')
  }else if(i <= 1 && i > 0.8){
    ra_re = c(ra_re, '0.8-1')
  }else if(i <= 0.8 && i > 0.5){
    ra_re = c(ra_re, '0.5-0.8')
  }else{
    ra_re = c(ra_re, '0-0.5')
  }
}

table(ra_re)
table(ra_95)

```